In [1]:
import logging

import pandas as pd
from xbbg import blp
from IPython.display import Markdown
from tulip.plots import plot_bar, plot_lines, switch_trace_to_secondary_axis
from tulip.data.dataquery import DataQueryClient as dq
from tulip.utils.notebook_related import display_two_charts

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    logger.setLevel(logging.CRITICAL + 1)


def transform_tenor(x):
    if x < 12:
        return ("Months", int(x))
    else:
        return ("Years", int(x / 12))


def style_scan(df):
    def hide_nan_css(v):
        return "visibility: hidden;" if pd.isna(v) else ""

    styler = df.style.background_gradient(axis=0)
    styler = styler.map(hide_nan_css).format(na_rep="", precision=2)
    return styler


def style_curve_frame(v):
    if pd.isna(v):
        return "background-color: white;"
    # Gradient from white (0) to black (20)
    v_clipped = max(0, min(20, v))
    grey = int(255 - (v_clipped / 20) * 255)
    bg_color = f"rgb({grey},{grey},{grey})"
    font_color = "white" if v_clipped > 12 else "black"
    if v < 0:
        font_color = "red"
    return f"background-color: {bg_color}; color: {font_color};"


# todo: probably move this to tulip
time_in_months_numeric = {
    "Today": 0,
    "1M": 1,
    "3M": 3,
    "6M": 6,
    "1Y": 12,
    "2Y": 24,
    "3Y": 36,
    "5Y": 60,
    "7Y": 84,
    "10Y": 120,
    "20Y": 240,
    "30Y": 360,
}


# Consolidated data loading
tenor = "1D"
countries = {
    "USD": "USA",
    "CAD": "CAN",
    "GBP": "GBR",
    "EUR": "EUR",
    "JPY": "JPN",
    "SEK": "SWE",
    "BRL": "BRA",
    "MXN": "MEX",
}

curves = {
    "AUD": "YCGT0001",  # AUD Australia Sovereign Curve
    "BRL": "YCGT0393",  # BRL Brazil Sovereign Curve
    "BGN": "YCGT0462",  # BGN Bulgaria Sovereign Curve
    "CAD": "YCGT0007",  # CAD Canada Sovereign Curve
    "CHF": "YCGT0082",  # CHF Switzerland Sovereign Curve
    "CNY": "YCGT0299",  # CNY China Sovereign Curve
    "COP": "YCGT0217",  # COP Colombia Sovereign Curve
    "CYP": "YCGT0261",  # CYP Cyprus Sovereign Curve (Former currency: Cypriot Pound)
    "CLP": "YCGT0351",  # CLP Chile Government Curve
    "DEM": "YCGT0016",  # DEM German Sovereign Curve (Former currency: Deutsche Mark)
    "DKK": "YCGT0011",  # DKK Denmark Sovereign Curve
    "EGP": "YCGT0811",  # EGP Egypt Sovereign Curve
    "ESP": "YCGT0061",  # ESP Spanish Sovereign Curve (Former currency: Spanish Peseta)
    "EUR": "YCGT0013",  # Euro Benchmarks Curve (Retained as 'EU' is not a country with a former national currency)
    "FIM": "YCGT0081",  # FIM Finland Sovereign Curve (Former currency: Finnish Markka)
    "FRF": "YCGT0014",  # FRF France Sovereign Curve (Former currency: French Franc)
    "GEL": "YCGT0803",  # GEL Republic of Georgia Sovereign Curve
    "GBP": "YCGT0022",  # GBP United Kingdom Sovereign Curve
    "GRD": "YCGT0156",  # GRD Greece Sovereign Curve (Former currency: Greek Drachma)
    "HKD": "YCGT0095",  # HKD Hong Kong Sovereign Curve
    "HUF": "YCGT0165",  # HUF Hungary Sovereign Curve
    "IDR": "YCGT0266",  # IDR Indonesia Sovereign Curve
    "IEP": "YCGT0062",  # IEP Ireland Sovereign Curve (Former currency: Irish Pound)
    "ILS": "YCGT0457",  # ILS Israel Sovereign - Galil Curve
    "INR": "YCGT0180",  # INR India Sovereign Curve
    "ISK": "YCGT0328",  # ISK Iceland Sovereign Curve
    "ITL": "YCGT0040",  # ITL Italy Sovereign Curve (Former currency: Italian Lira)
    "JPY": "YCGT0018",  # JPY Japan Sovereign Curve
    "KRW": "YCGT0173",  # KRW Korea Sovereign Curve
    "KZT": "YCGT0819",  # KZT Kazakhstan Sovereign Meukam Curve
    "LTL": "YCGT0341",  # LTL Lithuania Domestic Sovereign Curve (Former currency: Lithuanian Litas)
    "LVL": "YCGT0315",  # LVL Latvia Sovereign Curve (Former currency: Latvian Lats)
    "MXN": "YCGT0251",  # MXN Mexico Sovereign Curve
    "MYR": "YCGT0196",  # MYR Malaysia Sovereign Curve
    "NLG": "YCGT0020",  # NLG Netherlands Sovereign Curve (Former currency: Dutch Guilder)
    "NOK": "YCGT0078",  # NOK Norway Sovereign Curve
    "NZD": "YCGT0049",  # NZD New Zealand Sovereign Curve
    "PEN": "YCGT0361",  # PEN Peru Sovereign Curve
    "PLN": "YCGT0177",  # PLN Poland Sovereign Curve
    "PTE": "YCGT0084",  # PTE Portuguese Sovereign Curve (Former currency: Portuguese Escudo)
    "RON": "YCGT0508",  # RON Romanian Sovereign Curve
    "SEK": "YCGT0021",  # SEK Sweden Sovereign Curve
    "SIT": "YCGT0259",  # SIT Slovenia Sovereign Curve (Former currency: Slovenian Tolar)
    "SKK": "YCGT0256",  # SKK Slovakia Sovereign Curve (Former currency: Slovak Koruna)
    "THB": "YCGT0200",  # THB Thailand Sovereign Curve
    "TRL": "YCGT0250",  # TRL Turkey Sovereign Curve (Former currency: Turkish Lira - pre-2005, or sometimes used broadly for "Turkey Lira")
    "USD": "YCGT0025",  # US Treasury Actives Curve
    "VND": "YCGT0380",  # VND Vietnam Sovereign Curve
    "ZAR": "YCGT0090",  # ZAR South Africa Sovereign Curve
}

overnight_rates = {
    "USD": "SOFRRATE Index",  # United States – Fed Funds Effective Rate (US -> USD)
    "GBP": "SONIO/N Index",  # United Kingdom – SONIA Overnight Index (GB -> GBP)
    "EUR": "ESTRON Index",  # Euro Area – Euro Short-Term Rate (ESTR) (EU -> EUR, as there's no single 'former' currency for 'Euro Area' itself, ESTR is a post-euro rate)
    "JPY": "MUTSCALM Index",  # Japan – Uncollateralized Overnight Call Rate (JP -> JPY)
    "AUD": "RBACOR Index",  # Australia – Interbank Overnight Cash Rate (CORRA) (AU -> AUD)
    "CAD": "CAONREPO Index",  # Canada – Overnight Repo Rate (CA -> CAD)
    "SEK": "STIB1D Index",  # Sweden (SE -> SEK - Swedish Krona is still the currency)
    "BRL": "BZSELICA Index",  # Brazil (BR -> BRB, using an older/informal code for Brazilian Cruzeiro/Cruzado for historical context if needed, otherwise BRL is current)
    "KRW": "KWCR1T CMPN Curncy",  # Korean Call rate (KR -> KRW - Korean Won is still the currency)
    "CNY": "SHIF1W Index",  # Shibor 1 week (CN -> CNY - Chinese Yuan is still the currency, though could be RMB)
    "INR": "IN00O/N Index",  # Mibor Offered (IN -> INR - Indian Rupee is still the currency)
    "MXN": "MXIBTIEF Index",  # Mexico (MX -> MXP, for the old Mexican Peso before the 1993 revaluation)
    "ZAR": "SAONBOR Index",  # South Africa (ZA -> ZAR - South African Rand is still the currency, but GBP was its predecessor)
}


In [2]:
today = (
    blp.bdp(tickers=[overnight_rates[x] for x in countries.keys()], flds=["PX_LAST"])
    .rename(index={v: k for k, v in overnight_rates.items()})
    .squeeze()
    .rename("Today")
)

In [3]:
# Create wpi dictionary with (country, currency) tuple keys
wpi = {}
curve = {}

for ccy, ctry in countries.items():
    wpi[ccy] = dq.wpi(ccy, tenor=tenor)
    curve[ccy] = blp.bds(tickers=f"{curves[ccy]} Index", flds=["CURVE_TENOR_RATES"])

In [4]:
curve_frame = pd.concat(curve)
curve_frame.index.names = ["ccy", "curve"]
curve_frame = (
    curve_frame.droplevel("curve")
    .set_index("tenor", append=True)["mid_yield"]
    .unstack("tenor")
)
month_cols = sorted(
    [c for c in curve_frame.columns if c.endswith("M")], key=lambda x: int(x[:-1])
)
year_cols = sorted(
    [c for c in curve_frame.columns if c.endswith("Y")], key=lambda x: int(x[:-1])
)
curve_frame = curve_frame[month_cols + year_cols]

In [5]:
curve_frame["Today"] = today

curve_frame = curve_frame.loc[
    countries.keys(),
    ["Today", "1M", "3M", "6M", "1Y", "2Y", "3Y", "5Y", "7Y", "10Y", "20Y", "30Y"],
]


styled_curve_frame = curve_frame.style.format("{:.2f}").map(style_curve_frame)

In [6]:
styled_curve_frame

tenor,Today,1M,3M,6M,1Y,2Y,3Y,5Y,7Y,10Y,20Y,30Y
ccy,,,,,,,,,,,,
USD,3.95,3.94,3.88,3.80,3.65,3.57,3.56,3.67,3.85,4.07,4.64,4.67
CAD,2.27,2.21,2.19,2.25,2.31,2.45,2.52,2.74,2.92,3.14,3.47,3.58
GBP,3.97,4.10,3.96,3.87,3.68,3.72,3.73,3.87,4.01,4.40,5.06,5.19
EUR,1.93,1.97,2.02,2.04,2.07,2.00,2.05,2.25,2.38,2.64,3.12,3.23
JPY,0.48,0.44,0.43,0.60,0.72,0.93,1.01,1.24,1.48,1.68,2.69,3.18
SEK,1.75,nan,1.77,1.82,1.86,1.99,nan,2.30,2.39,2.67,nan,nan
BRL,14.90,nan,nan,nan,13.97,13.04,12.90,13.36,nan,13.62,nan,nan
MXN,7.26,-0.40,7.28,7.33,7.48,7.48,7.77,8.26,nan,8.92,9.44,9.51


In [7]:
discounting_df = (
    curve_frame.rename(columns=time_in_months_numeric)
    .T.interpolate("slinear")
    .round(3)
    .T
)


In [8]:
import numpy as np


def plot_discounting(curve_df, title="<b>Easing/Tightenings Priced In The Curve<b>"):
    fig = plot_lines(
        curve_df.sub(curve_df.iloc[:, 0], axis=0).T,
        title=title,
        show_0=True,
    )

    y_range = fig.layout.yaxis.range or [None, None]
    if y_range[0] is not None and y_range[1] is not None:
        y_ticks = np.arange(
            np.floor(y_range[0] * 2) / 2, np.ceil(y_range[1] * 2) / 2, 0.5
        )
        fig.update_yaxes(tickvals=y_ticks)
    else:
        y_ticks = fig.layout.yaxis.tickvals

    switch_trace_to_secondary_axis(fig, "USD")

    if 360 in curve_df.columns:
        x_ticks = ["" if k.endswith("M") else k for k in time_in_months_numeric.keys()]
    else:
        x_ticks = list(time_in_months_numeric.keys())

    fig.update_layout(
        yaxis=dict(
            tickvals=y_ticks,
            ticktext=[f"{val * 100:.0f}" for val in y_ticks],
            title="Absolute change (bps)",
        ),
        yaxis2=dict(
            anchor="y",
            range=y_range,
            overlaying="y",
            side="right",
            title="# of 25bps Hikes/Cuts",
            tickvals=y_ticks,
            ticktext=[f"{val / 0.25:.0f}" for val in y_ticks],
        ),
        xaxis=dict(
            title="Tenor",
            tickmode="array",
            tickvals=list(time_in_months_numeric.values()),
            ticktext=x_ticks,
        ),
    )
    return fig


fig_all = plot_discounting(discounting_df)
fig_sub_1y = plot_discounting(discounting_df.loc[:, discounting_df.columns <= 12])

### What Is Priced in IRS
This dashboard presents pricing expectations derived from Implied Rate Swaps sourced from JP Morgan. It is more comprehensive than Bloomberg's WIRP, though some pricing might be come from less liquid sources.


In [9]:
display_two_charts(fig_all, fig_sub_1y)

#### United States  

In [10]:
ctry = "USA"
ccy = "USD"
fig = plot_bar(
    wpi[ccy],
    title=f"<b>{ctry} | {ccy}</b>. Expectations for {tenor} rate from IRS",
    tick_suffix="%",
)
fig.show()

#### United Kingdom

In [11]:
ctry = "GBR"
ccy = "GBP"
fig = plot_bar(
    wpi[ccy],
    title=f"<b>{ctry} | {ccy}</b>. Expectations for {tenor} rate from IRS",
    tick_suffix="%",
)
fig.show()

#### Eurozone

In [12]:
ctry = ccy = "EUR"
fig = plot_bar(
    wpi[ccy],
    title=f"<b>{ctry} | {ccy}</b>. Expectations for {tenor} rate from IRS",
    tick_suffix="%",
)
fig.show()

#### Japan
Note: Japan might be being visualized incorrectly. As a stacked bar in some tenors. Ignore the second bar if that is what happened

In [13]:
ctry = "JPN"
ccy = "JPY"
plot_bar(
    wpi[ccy],
    title=f"<b>{ctry} | {ccy}</b>. Expectations for {tenor} rate from IRS",
    tick_suffix="%",
)

#### Sweden

In [14]:
ctry = "SWE"
ccy = "SEK"
plot_bar(
    wpi[ccy],
    title=f"<b>{ctry} | {ccy}</b>. Expectations for {tenor} rate from IRS",
    tick_suffix="%",
)

#### Brazil    

In [15]:
ctry = "BRA"
ccy = "BRL"
plot_bar(
    wpi[ccy],
    title=f"<b>{ctry} | {ccy}</b>. Expectations for {tenor} rate from IRS",
    tick_suffix="%",
)

#### Mexico    

In [16]:
ctry = "MEX"
ccy = "MXN"
plot_bar(
    wpi[ccy],
    title=f"<b>{ctry} | {ccy}</b>. Expectations for {tenor} rate from IRS",
    tick_suffix="%",
)

In [17]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:35_